In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import annotations

import logging

import numpy as np

from scipy.io import readsav

from mhsxtrapy.field2d import Field2dData, check_fluxbalance, FluxBalanceState, alpha_HS04
from mhsxtrapy.vis import (
    plot_magnetogram_2D,
    plot_magnetogram_3D,
    plot_dpressure_z,
    plot_ddensity_z,
    plot_dpressure_xy,
    plot_ddensity_xy,
)
from mhsxtrapy.field3d import calculate_magfield
from mhsxtrapy.b3d import WhichSolution

In [ ]:
"""
EXAMPLE FOR MAGNETIC FIELD EXTRAPOLATION FROM ANALYTICAL BOUNDARY CONDITION INSTANTIATED THROUGH .sav FILE AND USING HAGINO AND SAKURAI (2004) FOR DETERMINATION OF ALPHA VALUE

LOW AND LOU (1990) EXAMPLE 

Low & Lou (1990) describe a method to construct theoretical examples of semi-analytical non-linear force-free fields. 
The model is referred to as semi-analytical, as it is analytical except for the numerical solution of an ordinary differential equation in one coordinate direction.
The overall solution depends on parameters usually called n and m. For details we refer to the original paper (see Low and Lou 1990).
Because the full solution is known these fields can be used for testing the reliability and accuracy of non-linear force-free extrapolation methods by imposing 
full vector boundary conditions taken from the Low and Lou (1990) solution. As such, this method has been used recently for testing of non-linear force-free models 
in e.g. Schrijver et al. (2006) and Jarolim et al. (2023) and for testing of an MHS model in Zhu and Wiegelmann (2018).

Although our extrapolation methods only require boundary conditions for B_z, one can use information about the horizontal magnetic field components at z = 0 to constrain 
the constant alpha using the method of Hagino and Sakurai (2004). For the Low and Lou (1990) field all magnetic field components on the boundary are known and hence one 
can apply the method. The existence of such solutions is somewhat equivalent to vector magnetograph data being available instead of, exclusively, the line-of-sight magnetic
field component.

The solutions by Low and Lou (1990), which are simple in structure and axisymmetric, are constructed as follows: a dipole-like point source including field-aligned currents is 
placed at (0, 0, -l) below the origin of the physical coordinate system (x, y, z). Then, the local coordinate system (X, Y, Z), in which the dipole lies, is orientated at an 
angle to (x, y, z). The rotation is carried out in the X-Z-plane in relation to the x-z-plane, such that 0 < φ < π/2 is the angle between the z-axis and the Z-axis. Extrapolation
is carried out in the z > 0 domain, as the photosphere is taken to be located in the x-y-plane at z = 0.

The line-of-sight magnetogram used as the boundary condition to mimic the photospheric magnetic field in this example is a slice extracted from the non-linear force-free model described above. The
parameters n = 1,m = 1, l = 0.3 and φ = 0.47 have been used for this purpose (Low and Lou 1990), and lead to the magnetogram consisting of one sink and one extended source. The structure of this boundary
condition can be seen in Figure 7.8. The size of the bottom boundary domain corresponds to 3.2 Mm in both x- and y-direction with resolution of 0.04 Mm pixel^-1 in all directions, which is a pixel size
comparable, for example, to Sunrise/IMaX observations. The magnetic field strength on the boundary varies from -1792 G to 446 G.

The .sav file containing the boundary condition was provided by the ISSI team "Magnetohydrostatic Modeling of the Solar Atmosphere with New Datasets" (https://teams.issibern.ch/magnetohydrostaticsolaratmosphere/)
and can be found in the folder EXAMPLE-Low-Lou/data.
"""

In [ ]:
"""
LOAD DATA FROM FILE
"""

path = "data/Analytic_boundary_data.sav"

data = readsav(path, python_dict=True, verbose=True)

In [ ]:
"""
READ INFORMATION FROM FILE
"""

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

info_array = data.get("info_array")
logging.info(f"Info Array: {info_array}")


info_boundary = data.get("info_boundary")
logging.info(f"Info Boundary: {info_boundary}")


info_pixel = data.get("info_pixel")
logging.info(f"Info Pixel: {info_pixel}")


info_unit = data.get("info_unit")
logging.info(f"Info Unit: {info_unit}")

In [ ]:
"""
INSTANIATE RELEVANT PARAMETERS
"""

data_bz = data["b2dz5"] # Read magnetic field data in Gauss, see "Info Array" and "Info Unit" above.
data_bx = data["b2dx5"]
data_by = data["b2dy5"]

nx = data_bz.shape[1]
ny = data_bz.shape[0]

pz = 40.0 * 10**-3  # Convert pixelsize from km into Mm, see "Info Pixel" in previous cell.
px = 40.0 * 10**-3
py = 40.0 * 10**-3

xmin, ymin, zmin = 0.0, 0.0, 0.0
xmax = nx * px
ymax = ny * py
zmax = 4000.0 * 10**-3

nz = int(np.floor(zmax / pz))

nf = min(nx, ny)

x_arr = np.linspace(xmin, xmax, nx, dtype=np.float64)
y_arr = np.linspace(ymin, ymax, ny, dtype=np.float64)
z_arr = np.linspace(zmin, zmax, nz, dtype=np.float64)

In [ ]:
"""
CHECK FLUXBALANCE OF BOUNDARY CONDITION

check_fluxbalance returns the summation of the flux through the bottom boundary, normalised by the sum of absolute values. 
This returns a value between −1 and 1, where either extreme corresponds to the flux being directed entirely inward or entirely outward, respectively. We consider values 
between −0.01 and 0.01 as flux-balanced in agreement with Wiegelmann & Sakurai (2021). 
"""

np.fabs(check_fluxbalance(data_bz)) < 0.01

In [ ]:
"""
INSTANTIATE Field2dData OBJECT FROM LOW AND LOU BOUNDARY CONDITION
"""

data2d = Field2dData(
    nx,
    ny,
    nz,
    nf,
    px,
    py,
    pz,
    x_arr,
    y_arr,
    z_arr,
    data_bz,
    flux_balance_state=FluxBalanceState.BALANCED,
)

In [ ]:
"""
PLOT 2D BOUNDARY CONDITION
"""

plot_magnetogram_2D(data2d)

In [ ]:
"""
CALCULATE ALPHA BASED ON HAGINO AND SAKURAI (2004)

"Optimal" alpha calculated according to Hagino and Sakurai (2004). Alpha is calculated from the vertical electric current in the photosphere (from horizontal photospheric field) 
and the photospheric vertical magnetic field.
"""

alpha_HS04(data_bx, data_by, data_bz)

In [ ]:
"""
EXTRAPOLATE MAGNETIC FIELD
"""

data3d = calculate_magfield(
    data2d,
    alpha=-0.03,
    a=0.27,
    which_solution=WhichSolution.ASYMP,
    b=1.0,
    z0=2.0,
    deltaz=0.2,
) 

In [ ]:
"""
PLOT MAGNETOGRAM
"""

plot_magnetogram_3D(data3d, view='los', footpoints='active-regions')

In [ ]:
"""
PLOT PRESSURE VARIATION WITH HEIGHT z
"""

plot_dpressure_z(data3d)

In [ ]:
"""
PLOT DENSITY VARIATION WITH HEIGHT z
"""

plot_ddensity_z(data3d)

In [ ]:
"""
PLOT PRESSURE VARIATION FOR A SPECIFIC HEIGHT z [Mm] FOR ALL x and y
"""

plot_dpressure_xy(
    data3d, 0
)

In [ ]:
"""
PLOT DENSITY VARIATION FOR A SPECIFIC HEIGHT z [Mm] FOR ALL x and y
"""

plot_ddensity_xy(
    data3d, 0
)